In [1]:
from __future__ import print_function  #We'll talk about these imports later.
import networkx as nx
import numpy as np
import scipy as sp
import pickle
from scipy import ndimage
import fugu
from fugu import Scaffold, Brick
from fugu.bricks import Vector_Input, Threshold, Dot

# How to find a good taco using Fugu

Fugu is/will be useful for all sorts of data processing on heterogenous platforms, from large HPC systems to embedded and edge devices.  Here, we run through a classic issue: Finding a tasty taco.

Sarah is sitting at home hungry and wants to go to her favorite taco shop.  Unfortunately, they've recently moved locations and she doesn't know where their new spot is.  

Luckily, Sarah works for a national lab and has access to newly updated satellite imagery.  Here's the image she has:

![](OverheadMap.png)

Now, Sarah could carefully scan the image herself looking for the taco shop.  However, that takes a long time and the image is pretty big!

So, instead, we'll use a simple template-matching classifier and fugu. Here's Sarah's reference image (coded into a 0,1-vector of spikes).

In [2]:
taco_img = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,],
[0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,],
[0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,],
[0,0,0,0,0,0,0,1,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,],
[0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,],
[0,0,0,0,0,0,1,1,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,],
[0,0,0,0,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,],
[0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,],
[0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,],
[0,0,0,0,1,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,],
[0,0,0,0,1,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,],
[0,0,0,0,1,1,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]]
taco_img = np.array(taco_img).flatten().astype('float')

The images pulled from the satellite are conveniently stored in a dictionary in a numpy file.  We can load them easily:

In [3]:
collected_images = np.load('imgs.npy', allow_pickle=True)[()]  #The strange slicing [()] is because we've stored a dictionary within a numpy array. Convenient, but weird.

We're almost ready to go.  Let's look at those imports we skipped before...

```python
from __future__ import print_function
import networkx as nx
import numpy as np
import fugu
from fugu import Scaffold, Spike_Input, Threshold, Dot
```

`print_function` allows us to print in Python 2 using Python 3 style function calls.

NetworkX actually doesn't need to be imported here, but you'll quickly run into issues if you start playing around the with objects without it.

Numpy is a standard import since that provides our array-handling.

Here's a quick run-down of the fugu objects we use here:
- `Scaffold`:  The scaffold object is a container/graph object that organizes and holds computational nodes called 'bricks'.  Various types of bricks can be combined to construct a more sophisticated algorithm.  `Scaffold` is the main object for end-users.  The general workflow is something like this:
    1. Define a `Scaffold` object
    2. Attach bricks to the `Scaffold`, linking them together
    3. 'Build' the computational graph by calling `Scaffold.lay_bricks`
    4. Call `Scaffold.evaluate` and perform any post-processing to your answer.
A few todo-type caveats:  
    1. The `Scaffold` object should, itself, be a `Brick`.  This is critical but unfortunately currently missing.
    2. Inputs that take both pre-computed values and input spikes may not lay correctly.
    3. The code is missing a large portion of expected 'checks' so right now it is on the user to determine if they are doing something allowed or disallowed.

- `Spike_Input`: A `Brick` that provides spike inputs in the form of vectors.  For a single input, a numpy array (as you see below) is sufficient.  For inputs at various timesteps, use the flag `time_dimension`.  For generic vector input, use `coding=Raster`, though the user can specify an output coding as desired.
- `Dot`: A `Brick` that provides a dot product of two vectors.  While this could be updated in the future, right now the only implementation is uses weights, and the analog value is the output of the brick. This is referred to as `current` output coding (to signify threshold operation).  You can then link this with other bricks that accept `current` to produce spikes.  Understand that we cannot do very much with a signal when it is in `current` coding.  This responsiblilty is left to the programmer.  Easy things to do would be, for example, add or multiply (we'd be able to write these bricks).
- `Threshold`:  Another `Brick`.  Provides a basic thresholding.  Useful, for example, to perform a threshold which converts `current` coding into spikes. 


We begin to build our scaffold:

In [4]:
scaffold = Scaffold()
scaffold.add_brick(Vector_Input(np.array(taco_img), coding='Raster', name='Reference_Image'), 'input')

When we add a `Brick` to a `Scaffold`, by default, it adds it to the last node on the graph (generally, the last brick added).  Most bricks provide one ouput channel; separate output channels allow for a brick to send separate output signals to separate bricks. Below, we connect a different `Dot` brick to each of these pair (Reference_Image <> Test_Image); all on channel 0 (the first channel).  By default, a brick connects to the first output channel of a node if none is specified.  We also name them for fun and clarity.  We can check our progress using `scaffold.summary()`, but we see that not much is going on yet.

In [5]:
scaffold.summary()

Scaffold is built: False
-------------------------------------------------------
List of Bricks:


Brick No.: 0
Brick Tag: Reference_Image-0
Brick Name: Reference_Image
{'tag': 'Reference_Image-0', 'name': 'Reference_Image', 'brick': <fugu.bricks.input_bricks.Vector_Input object at 0x7fa70a8d2ac0>, 'layer': 'input', 'input_nodes': [(-2, 0)]}
Brick is built: False


-------------------------------------------------------


-------------------------------------------------------
List of Brick Edges:




Let's connect some bricks!  (In the future, we can suppport more complicated classifiers.  Here, we're doing a simple template match.)

In [6]:
threshold_value = 1000  #Number of positive pixels that have to agree for us to consider it a 'match'
for image_key in collected_images:
    test_image = collected_images[image_key].flatten() #We squeeze the 2-d image into a 1-d vector
    scaffold.add_brick(Dot(test_image, name='Dot_with_' + str(image_key)), (0,0))  #Connects this Dot brick to node 0, channel 0 (The input)
    scaffold.add_brick(Threshold(threshold_value, name='Does_Match_' + str(image_key)), output=True)  #We're using the fact that this'll connect to the last-added brick
scaffold.summary()

Scaffold is built: False
-------------------------------------------------------
List of Bricks:


Brick No.: 0
Brick Tag: Reference_Image-0
Brick Name: Reference_Image
{'tag': 'Reference_Image-0', 'name': 'Reference_Image', 'brick': <fugu.bricks.input_bricks.Vector_Input object at 0x7fa70a8d2ac0>, 'layer': 'input', 'input_nodes': [(-2, 0)]}
Brick is built: False


Brick No.: 1
Brick Tag: Dot_with_0-1
Brick Name: Dot_with_0
{'tag': 'Dot_with_0-1', 'name': 'Dot_with_0', 'brick': <fugu.bricks.utility_bricks.Dot object at 0x7fa70c7afd30>, 'input_nodes': [(0, 0)]}
Brick is built: False


Brick No.: 2
Brick Tag: Does_Match_0-2
Brick Name: Does_Match_0
{'tag': 'Does_Match_0-2', 'name': 'Does_Match_0', 'brick': <fugu.bricks.stochastic_bricks.Threshold object at 0x7fa70baccac0>, 'input_nodes': [], 'layer': 'output'}
Brick is built: False


Brick No.: 3
Brick Tag: Dot_with_1-3
Brick Name: Dot_with_1
{'tag': 'Dot_with_1-3', 'name': 'Dot_with_1', 'brick': <fugu.bricks.utility_bricks.Dot object at

In [7]:
#If you have matplotlib installed, you can plot the circuit using NetworkX's built in tools, e.g.:
nx.draw_kamada_kawai(scaffold.circuit)

ImportError: Matplotlib required for draw()

Keep in mind that, right now, the only thing that's important is the NetworkX Digraph `scaffold.circuit` and that it contains all the needed to build the network graph.

You can always access this directly which helps immensly with debugging.

In [8]:
#To get node (brick) information
print([scaffold.circuit.nodes[node] for node in scaffold.circuit.nodes])

[{'tag': 'Reference_Image-0', 'name': 'Reference_Image', 'brick': <fugu.bricks.input_bricks.Vector_Input object at 0x7fa70a8d2ac0>, 'layer': 'input', 'input_nodes': [(-2, 0)]}, {'tag': 'Dot_with_0-1', 'name': 'Dot_with_0', 'brick': <fugu.bricks.utility_bricks.Dot object at 0x7fa70c7afd30>, 'input_nodes': [(0, 0)]}, {'tag': 'Does_Match_0-2', 'name': 'Does_Match_0', 'brick': <fugu.bricks.stochastic_bricks.Threshold object at 0x7fa70baccac0>, 'input_nodes': [], 'layer': 'output'}, {'tag': 'Dot_with_1-3', 'name': 'Dot_with_1', 'brick': <fugu.bricks.utility_bricks.Dot object at 0x7fa70bacca30>, 'input_nodes': [(0, 0)]}, {'tag': 'Does_Match_1-4', 'name': 'Does_Match_1', 'brick': <fugu.bricks.stochastic_bricks.Threshold object at 0x7fa70bacca60>, 'input_nodes': [], 'layer': 'output'}, {'tag': 'Dot_with_2-5', 'name': 'Dot_with_2', 'brick': <fugu.bricks.utility_bricks.Dot object at 0x7fa70bacca90>, 'input_nodes': [(0, 0)]}, {'tag': 'Does_Match_2-6', 'name': 'Does_Match_2', 'brick': <fugu.bricks

We're now ready to evaluate the functions and get an answer.

`Scaffold.lay_bricks` performs all the necessary steps to ready a full graph of neurons (not just computational nodes) for a hardware platform or simulator.  After we build it, we see that the built network graph (i.e. `scaffold.graph`) is much a larger. (You can click left of the output cell to expand/collapse.)

In [9]:
graph = scaffold.lay_bricks()  #The return here is not needed, but useful if you want to access the graph easily.  You can always use scaffold.graph instead (assuming it exists).
scaffold.summary()

ValueError: Only one input is permitted.

A little description of what is going on in the 'list of neurons':  'threshold', 'decay', and 'p' are all neuron parameters used by the the evaluation platform (e.g. the simulator).  'index' is a tuple describing the local index of that neuron relative to its encoding scheme.  So, for example, if you have a two-dimensional, three-bit binary encoding, your shape might be (3,2) and so an index of (1,2) represents the 1st bit (counting from the 0th bit) of the 2nd number.  

We can now evaluate the computation.  Right now, we only support the ds simulator. It should be in the git repot, if not just copy ds.py into your working directory. 

In [11]:
result = scaffold.evaluate(backend='ds')

Now, if we print the result, it'll probably be a little disappointing:

In [12]:
print(result)

   time  neuron_number
0   1.0         2511.0


All this says is that in timestep 1, neuron 2511 spiked. No other neurons spiked...

We need to add helper functions to decode to spike values.  Since there's an associated output coding for all outputs, this shouldn't be too hard.  It's just low on the priorities list since, for now, everyone using Fugu can read a spike raster.

The following code will tell us what neuron spiked:

In [13]:
[node + str(':') +  str(scaffold.graph.nodes[node]) for node in scaffold.graph.nodes][2511]

"Does_Match_4:{'threshold': 1000, 'decay': 0.0, 'p': 1.0, 'brick': 'Does_Match_4', 'neuron_number': 2511, 'record': ['spikes']}"

So, it turns out that `collected_images[4]` is the taco shop!  Great!

But wait!  Sarah only has 15 minutes before heading to work...

So, the question is can she make it to the taco shop in time? We'll need another brick and another look at that map.

In [14]:
from fugu.bricks import Shortest_Path, Concatenate

Here's the map with node numbers assigned:

![](OverheadMap2.png)


We could just look at all the ways to get to the taco shop, but on a large graph like this, it can be hard to find a short path (especially with all the traffic near town hall).

And, being a national lab employee, Sarah doesn't just want a short path; she wants the shortest path! Fugu to the rescue...

In [15]:
#Helperfunction to constuct the map.
#Sarah already has a copy since she gets tacos often.
def generate_town_map():
    town_map = nx.Graph()
    town_map.add_node(0, label='Home')
    town_map.add_edge(0,1,weight=2)
    town_map.add_edge(0,7,weight=7)
    town_map.add_edge(1,2,weight=1)
    town_map.add_edge(1,6,weight=15)
    town_map.add_edge(2,3,weight=6)
    town_map.add_edge(2,5,weight=18)
    town_map.add_edge(3,4,weight=8)
    town_map.add_edge(4,5,weight=1)
    town_map.add_edge(5,6,weight=4)
    town_map.add_edge(6,7,weight=2)
    return town_map

In [16]:
town_map = generate_town_map()

We could create a scaffold to determine if Sarah can make it to lunch using the information we got from our previous scaffold.

But, fugu is designed to be compositionable, so we might as well make it one large graph.

The only tricky bit here is that we need to use a `Concatenate` brick because `Shortest_Path_Length` is looking for a single input (rather than a distinct input for each node).

In [18]:
threshold_value = 1000  #Number of positive pixels that have to agree for us to consider it a 'match'
new_scaffold = Scaffold()
new_scaffold.add_brick(Vector_Input(np.array(taco_img), coding='Raster', name='Reference_Image'), 'input')
for image_key in collected_images:
    test_image = collected_images[image_key].flatten() #We squeeze the 2-d image into a 1-d vector
    #scaffold.add_brick(Spike_Input(np.array(test_image),coding='Raster', name='Test_Image-' + str(image_key)),'input')
    new_scaffold.add_brick(Dot(test_image, name='Dot_with_' + str(image_key)), (0,0))  #Connects this Dot brick to node 0, channel 0 (The input)
    new_scaffold.add_brick(Threshold(threshold_value, name='Does_Match_' + str(image_key)))  #We no longer want outputs from these bricks
new_scaffold.add_brick(Concatenate(), ['Does_Match_'+str(image_key) for image_key in collected_images])
new_scaffold.add_brick(Shortest_Path(town_map, 0, name='Shortest_Path')) #Shortest_Path_Length requires a target_graph and a target_node
new_scaffold.add_brick(Threshold(15, name='Final_Output'),output=True)  #We want to know this final check

In [19]:
new_scaffold.lay_bricks()
result = new_scaffold.evaluate(max_runtime=100,backend='ds',record_all=True) #We'll record all the neurons this time for some checking

In [20]:
#Let's decode the result
#Get the output brick index
output_brick_index = [new_scaffold.circuit.nodes[node]['name'] for node in new_scaffold.circuit.nodes].index('Final_Output')

#Get the complete node from the output brick
complete = new_scaffold.circuit.nodes[output_brick_index]['control_nodes'][0]['complete']

#Get the output neuron from the output brick
output = new_scaffold.circuit.nodes[output_brick_index]['output_lists'][0][0]

In [21]:
#Search to see if output neuron ever spiked

def check_for_spikes(neuron_name):
    #Get the names for all the neurons
    #Then find the output neuron
    output_neuron_index = [node for node in new_scaffold.graph.nodes].index(neuron_name)
    found_a_spike = False
    when = None
    for index, row in result.iterrows():
        timestep = row['time']
        neuron = row['neuron_number']
        if output_neuron_index == neuron:
            found_a_spike=True
            when = timestep
    return found_a_spike, when

(found_spike, when) = check_for_spikes(output)
if found_spike:
    print("Yes, Neuron " + str(output) + " spiked at time" + str(when) + ".")
else:
    print("No, Sarah made it in time.")

No, Sarah made it in time.


Wait. What if the algorithm just didn't have time to finish.  Can we check that the final brick did indeed finish?

In [23]:
(found_spike, when) = check_for_spikes(complete)
if found_spike:
    print("Yes, Neuron " + str(complete) + " spiked at time " + str(when) + ".")
else:
    print("No, we never finished the algorithm.")

Yes, Neuron Final_Output_complete spiked at time 18.0.


In [24]:
def create_images():
    collected_images = dict()
    filenames = ['coffee.png', 'home.png', 'pizza.png', 'store.png', 'taco.png', 'townhall.png']
    for  i, filename in enumerate(filenames):
        taco_img = ndimage.imread(filename,mode='P')
        taco_img = -1*(np.round(sp.misc.imresize(taco_img,(50,50))/255).astype('int')-1)
        collected_images[i] = taco_img
    np.save('imgs.npy', collected_images)
def print_img(taco_img):
    taco_img = np.array(taco_img)
    for line in range(taco_img.shape[0]):
        print('[',end='')
        for pixel in range(taco_img.shape[1]):
            print(taco_img[line,pixel],end=',')
        print('],')